# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorkuta,67.5000,64.0000,266.78,93,59,6.54,RU,1682639874
1,1,shima,24.4478,117.8111,293.16,74,10,0.35,CN,1682639938
2,2,waitangi,-43.9535,-176.5597,286.38,78,96,1.34,NZ,1682639798
3,3,adamstown,-25.0660,-130.1015,297.10,66,96,2.57,PN,1682639868
4,4,puerto ayora,-0.7393,-90.3518,300.12,89,70,2.24,EC,1682639876


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity= city_data_df["Humidity"]

map_plot=city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", size="Humidity", color= "city", scale=0.5)

# Display the map
map_plot

ImportError: Geographic projection support requires GeoViews and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df["Cloudiness"]==0]

city_data_df= city_data_df[(city_data_df["Max Temp"]>295)]

# Drop any rows with null values
city_data_df=city_data_df.dropna()

city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,nalut,30.3333,10.8500,295.86,22,0,6.32,LY,1682639939
24,24,lihue,21.9789,-159.3672,301.73,65,0,6.17,US,1682639942
26,26,ubari,26.5921,12.7805,297.32,16,0,3.79,LY,1682639943
65,65,pacific grove,36.6177,-121.9166,298.51,73,0,4.02,US,1682639953
90,90,brandon,27.9378,-82.2859,301.64,70,0,5.14,US,1682639959
91,91,pisco,-13.7000,-76.2167,295.18,83,0,7.72,PE,1682639835
152,152,crane,31.3974,-102.3501,299.12,15,0,2.29,US,1682639976
156,156,cabo san lucas,22.8909,-109.9124,301.82,47,0,6.69,MX,1682639702
162,162,vangviang,18.9333,102.4500,299.12,74,0,0.61,LA,1682639979
172,172,east wenatchee bench,47.4257,-120.2812,297.78,27,0,4.12,US,1682639981


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country","Lng", "Lat", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
7,nalut,LY,10.8500,30.3333,22,
24,lihue,US,-159.3672,21.9789,65,
26,ubari,LY,12.7805,26.5921,16,
65,pacific grove,US,-121.9166,36.6177,73,
90,brandon,US,-82.2859,27.9378,70,
91,pisco,PE,-76.2167,-13.7000,83,
152,crane,US,-102.3501,31.3974,15,
156,cabo san lucas,MX,-109.9124,22.8909,47,
162,vangviang,LA,102.4500,18.9333,74,
172,east wenatchee bench,US,-120.2812,47.4257,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":  geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc [index, "Lat"]
    long = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{latitude},{radius}"
    params["bias"] = f"proximity:{long},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nalut - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
ubari - nearest hotel: فندق اوباري
pacific grove - nearest hotel: Pacific Grove Inn
brandon - nearest hotel: Brandon Motor Lodge
pisco - nearest hotel: La Portada
crane - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
vangviang - nearest hotel: SD View Hotel
east wenatchee bench - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
medina - nearest hotel: The Oberoi Madina
prado - nearest hotel: Pousada Recanto do Prado
toliara - nearest hotel: Ambary
hutchinson island south - nearest hotel: Vistana Beach Club 1
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
al kharijah - nearest hotel: Kharga Hotel
tucumcari - nearest hotel: Pow Wow Inn
ascension - nearest hotel: Cajun Country Inn
fortaleza - nearest hotel: Hotel Catedral
villa juarez - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found


,City,Country,Lng,Lat,Humidity,Hotel Name
7,nalut,LY,10.8500,30.3333,22,No hotel found
24,lihue,US,-159.3672,21.9789,65,Kauai Palms
26,ubari,LY,12.7805,26.5921,16,فندق اوباري
65,pacific grove,US,-121.9166,36.6177,73,Pacific Grove Inn
90,brandon,US,-82.2859,27.9378,70,Brandon Motor Lodge
91,pisco,PE,-76.2167,-13.7000,83,La Portada
152,crane,US,-102.3501,31.3974,15,No hotel found
156,cabo san lucas,MX,-109.9124,22.8909,47,Comfort Rooms
162,vangviang,LA,102.4500,18.9333,74,SD View Hotel
172,east wenatchee bench,US,-120.2812,47.4257,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
mapplot= hotel_df.hvplot.points("Lat", "Lng", geo=True, tiles="OSM", size="Humidity", color= "city", scale=1, frame_width= 700, frame_height =600, hover_cols= ["Hotel Name", "Country"])

# Display the map
mapplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)